In [10]:
from utils import *
from os import listdir, makedirs
from os.path import isfile, join, exists
import os
import rsa

granule = 'ATL03_20210715182907_03381203_005_01.h5'
shapefile = '/shapefiles/jakobshavn_small.shp'
gtxs = 'all'
datadir = '/IS2data'

In [25]:
def download_granule_nsidc(granule_id, gtxs, shapefile, granule_output_path, uid, pwd): 
    """
    Download a single ICESat-2 ATL03 granule based on its producer ID,
    subsets it to a given shapefile, and puts it into the specified
    output directory as a .h5 file. A NASA earthdata user id (uid), and
    the associated password are required. 

    Parameters
    ----------
    granule_id : string
        the producer_granule_id for CMR search
    gtxs : string or list
        the ground tracks to request
        possible values:
            'gt1l' or 'gt1r' or 'gt2l', ... (single gtx)
            ['gt1l', 'gt3r', ...] (list of gtxs)
    shapefile : string
        filepath to the shapefile used for spatial subsetting
    granule_output_path : string
        folder in which to save the subsetted granule
    uid : string
        earthdata user id
    pwd : string
        the associated password

    Returns
    -------
    nothing

    Examples
    --------
    >>> download_granule_nsidc(granule_id='ATL03_20210715182907_03381203_005_01.h5', 
                               shapefile='/shapefiles/jakobshavn.shp', 
                               gtxs='gt1l'
                               granule_output_path='/IS2data', 
                               uid='myuserid', 
                               pwd='mypasword')
    """
    
    import requests
    import json
    import zipfile
    import os
    import shutil
    import re
    import geopandas as gpd
    from shapely.geometry import Polygon, mapping
    from shapely.geometry.polygon import orient
    from xml.etree import ElementTree as ET
    import numpy as np
    
    short_name = 'ATL03'
    version = granule_id[30:33]
    granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'
    capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{short_name}.{version}.xml'
    base_url = 'https://n5eil02u.ecs.nsidc.org/egi/request'
    
    shapefile_filepath = str(os.getcwd() + shapefile)
    
    # set the variables for subsetting
    vars_sub = ['/ancillary_data/atlas_sdp_gps_epoch',
                '/orbit_info/rgt',
                '/orbit_info/cycle_number',
                '/orbit_info/sc_orient',
                '/gtx/geolocation/ph_index_beg',
                '/gtx/geolocation/segment_dist_x',
                '/gtx/geolocation/segment_length',
                '/gtx/geophys_corr/dem_h',
                '/gtx/geophys_corr/geoid',
                '/gtx/bckgrd_atlas/pce_mframe_cnt',
                '/gtx/bckgrd_atlas/bckgrd_counts',
                '/gtx/bckgrd_atlas/bckgrd_int_height',
                '/gtx/bckgrd_atlas/delta_time',
                '/gtx/heights/lat_ph',
                '/gtx/heights/lon_ph',
                '/gtx/heights/h_ph',
                '/gtx/heights/dist_ph_along',
                '/gtx/heights/delta_time',
                '/gtx/heights/pce_mframe_cnt',
                '/gtx/heights/quality_ph']
    beam_list = ['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']
    
    if gtxs == 'all':
        var_list = sum([[v.replace('/gtx','/'+bm) for bm in beam_list] if '/gtx' in v else [v] for v in vars_sub],[])
    elif type(gtxs) == str:
        var_list = [v.replace('/gtx','/'+gtxs.lower()) if '/gtx' in v else v for v in vars_sub]
    elif type(gtxs) == list:
        var_list = sum([[v.replace('/gtx','/'+bm.lower()) for bm in gtxs] if '/gtx' in v else [v] for v in vars_sub],[])
    else: # default to requesting all beams
        var_list = sum([[v.replace('/gtx','/'+bm) for bm in beam_list] if '/gtx' in v else [v] for v in vars_sub],[])
    
    # search for the given granule
    search_params = {
        'short_name': short_name,
        'page_size': 100,
        'page_num': 1,
        'producer_granule_id': granule_id}

    granules = []
    headers={'Accept': 'application/json'}
    while True:
        response = requests.get(granule_search_url, params=search_params, headers=headers)
        results = json.loads(response.content)

        if len(results['feed']['entry']) == 0:
            # Out of results, so break out of loop
            break

        # Collect results and increment page_num
        granules.extend(results['feed']['entry'])
        search_params['page_num'] += 1

    print('\nDownloading ICESat-2 data. Found granules:')
    for result in granules:
        print('  '+result['producer_granule_id'], f', {float(result["granule_size"]):.2f} MB',sep='')
        
    # Use geopandas to read in polygon file as GeoDataFrame object 
    # Note: a KML or geojson, or almost any other vector-based spatial data format could be substituted here.
    gdf = gpd.read_file(shapefile_filepath)
    
    # Simplify polygon for complex shapes in order to pass a reasonable request length to CMR. 
    # The larger the tolerance value, the more simplified the polygon.
    # Orient counter-clockwise: CMR polygon points need to be provided in counter-clockwise order. 
    # The last point should match the first point to close the polygon.
    poly = orient(gdf.simplify(0.05, preserve_topology=False).loc[0],sign=1.0)

    geojson = gpd.GeoSeries(poly).to_json() # Convert to geojson
    geojson = geojson.replace(' ', '') #remove spaces for API call
    
    #Format dictionary to polygon coordinate pairs for CMR polygon filtering
    polygon = ','.join([str(c) for xy in zip(*poly.exterior.coords.xy) for c in xy])
    
    print('\nInput shapefile:', shapefile)
    print('Simplified polygon coordinates based on shapefile input:', polygon)
    
    # Create session to store cookie and pass credentials to capabilities url
    session = requests.session()
    s = session.get(capability_url)
    response = session.get(s.url,auth=(uid,pwd))
    
    
    
    
    print('\n\n', uid)
    print(pwd)
    print(capability_url)
    print(s.url)
    print(response)
    print(response.content, '\n\n')
    '''
    root = ET.fromstring(response.content)

    #collect lists with each service option
    subagent = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]
    
    # this is for getting possible variable values from the granule search
    if len(subagent) > 0 :
        # variable subsetting
        variables = [SubsetVariable.attrib for SubsetVariable in root.iter('SubsetVariable')]  
        variables_raw = [variables[i]['value'] for i in range(len(variables))]
        variables_join = [''.join(('/',v)) if v.startswith('/') == False else v for v in variables_raw] 
        variable_vals = [v.replace(':', '/') for v in variables_join]
    
    # make sure to only request the variables that are available
    def intersection(lst1, lst2):
        lst3 = [value for value in lst1 if value in lst2]
        return lst3
    var_list_subsetting = intersection(variable_vals,var_list)
    
    if len(subagent) < 1 :
        print('No services exist for', short_name, 'version', latest_version)
        agent = 'NO'
        coverage,Boundingshape = '',''
    else:
        agent = ''
        subdict = subagent[0]
        if subdict['spatialSubsettingShapefile'] == 'true':
            Boundingshape = geojson
        else:
            Boundingshape = ''
        coverage = ','.join(var_list_subsetting)
    '''
    agent = ''
    Boundingshape = geojson
    Boundingshape = ''
    coverage = ','.join(vars_sub)
    coverage = ''
    
    
    
        
    page_size = 100
    request_mode = 'stream'
    page_num = int(np.ceil(len(granules)/page_size))

    param_dict = {'short_name': short_name, 
                  'producer_granule_id': granule_id,
                  'version': version,  
                  'polygon': polygon,
                  'Boundingshape': Boundingshape,  
                  'Coverage': coverage, 
                  'page_size': page_size, 
                  'request_mode': request_mode, 
                  'agent': agent, 
                  'email': 'yes'}

    #Remove blank key-value-pairs
    param_dict = {k: v for k, v in param_dict.items() if v != ''}

    #Convert to string
    param_string = '&'.join("{!s}={!r}".format(k,v) for (k,v) in param_dict.items())
    param_string = param_string.replace("'","")

    #Print API base URL + request parameters
    endpoint_list = [] 
    for i in range(page_num):
        page_val = i + 1
        API_request = api_request = f'{base_url}?{param_string}&page_num={page_val}'
        endpoint_list.append(API_request)

    print('\nAPI request URL:')
    print(*endpoint_list, sep = "\n") 
    
    # Create an output folder if the folder does not already exist.
    path = str(os.getcwd() + granule_output_path)
    if not os.path.exists(path):
        os.mkdir(path)

    # Different access methods depending on request mode:
    for i in range(page_num):
        page_val = i + 1
        print('\nOrder: ', page_val)
        print('Requesting...')
        request = session.get(base_url, params=param_dict)
        print('HTTP response from order response URL: ', request.status_code)
        request.raise_for_status()
        d = request.headers['content-disposition']
        fname = re.findall('filename=(.+)', d)
        dirname = os.path.join(path,fname[0].strip('\"'))
        print('Downloading...')
        open(dirname, 'wb').write(request.content)
        print('Data request', page_val, 'is complete.')

    # Unzip outputs
    for z in os.listdir(path): 
        if z.endswith('.zip'): 
            zip_name = path + "/" + z 
            zip_ref = zipfile.ZipFile(zip_name) 
            zip_ref.extractall(path) 
            zip_ref.close() 
            os.remove(zip_name) 

    # Clean up Outputs folder by removing individual granule folders 
    for root, dirs, files in os.walk(path, topdown=False):
        for file in files:
            try:
                shutil.move(os.path.join(root, file), path)
            except OSError:
                pass
        for name in dirs:
            os.rmdir(os.path.join(root, name))
            
    print('\nUnzipped files and cleaned up directory.')
    print('Output data saved in:', granule_output_path)
            
    return

In [26]:
download_granule_nsidc(granule, gtxs, shapefile, datadir, decedc(edc().u), decedc(edc().p))


  ATL03_20210715182907_03381203_005_01.h5, 5065.14 MB

Input shapefile: /shapefiles/jakobshavn_small.shp
Simplified polygon coordinates based on shapefile input: -49.59098555896507,69.10415896488335,-50.33805587146507,68.42374368072217,-50.01945235584007,68.20452489563448,-48.47038009021507,68.14325568125464,-48.16276290271507,68.66085965388693,-48.12980391834007,69.0963205694427,-49.59098555896507,69.10415896488335


 arndtp
My5ecretPw0rd!
https://n5eil02u.ecs.nsidc.org/egi/capabilities/ATL03.005.xml
https://uat.urs.earthdata.nasa.gov/oauth/authorize?app_type=401&client_id=PGVMJ5nUzSnQkI5o23gMxA&response_type=code&redirect_uri=https%3A%2F%2Fn5eil02u.ecs.nsidc.org%2FOPS%2Fredirect&state=aHR0cHM6Ly9uNWVpbDAydS5lY3MubnNpZGMub3JnL2VnaS9jYXBhYmlsaXRpZXMvQVRMMDMuMDA1LnhtbA
<Response [400]>
b'{"error":"invalid_request"}' 



API request URL:
https://n5eil02u.ecs.nsidc.org/egi/request?short_name=ATL03&producer_granule_id=ATL03_20210715182907_03381203_005_01.h5&version=005&polygon=-49.5909855

HTTPError: 400 Client Error: Bad Request for url: https://uat.urs.earthdata.nasa.gov/oauth/authorize?app_type=401&client_id=PGVMJ5nUzSnQkI5o23gMxA&response_type=code&redirect_uri=https%3A%2F%2Fn5eil02u.ecs.nsidc.org%2FOPS%2Fredirect&state=aHR0cHM6Ly9uNWVpbDAydS5lY3MubnNpZGMub3JnL2VnaS9yZXF1ZXN0P3Nob3J0X25hbWU9QVRMMDMmcHJvZHVjZXJfZ3JhbnVsZV9pZD1BVEwwM18yMDIxMDcxNTE4MjkwN18wMzM4MTIwM18wMDVfMDEuaDUmdmVyc2lvbj0wMDUmcG9seWdvbj0tNDkuNTkwOTg1NTU4OTY1MDclMkM2OS4xMDQxNTg5NjQ4ODMzNSUyQy01MC4zMzgwNTU4NzE0NjUwNyUyQzY4LjQyMzc0MzY4MDcyMjE3JTJDLTUwLjAxOTQ1MjM1NTg0MDA3JTJDNjguMjA0NTI0ODk1NjM0NDglMkMtNDguNDcwMzgwMDkwMjE1MDclMkM2OC4xNDMyNTU2ODEyNTQ2NCUyQy00OC4xNjI3NjI5MDI3MTUwNyUyQzY4LjY2MDg1OTY1Mzg4NjkzJTJDLTQ4LjEyOTgwMzkxODM0MDA3JTJDNjkuMDk2MzIwNTY5NDQyNyUyQy00OS41OTA5ODU1NTg5NjUwNyUyQzY5LjEwNDE1ODk2NDg4MzM1JnBhZ2Vfc2l6ZT0xMDAmcmVxdWVzdF9tb2RlPXN0cmVhbSZlbWFpbD15ZXM

In [ ]:
filelist = [datadir[1:]+'/'+f for f in listdir(datadir[1:]) if isfile(join(datadir[1:], f)) & ('.h5' in f)]
print('\nNumber of processed ATL03 granules to read in: ' + str(len(filelist)))
    
photon_data, bckgrd_data, ancillary = read_atl03(filelist[0], geoid_h=True)
print_granule_stats(photon_data, bckgrd_data, ancillary, outfile='stats.txt')

In [ ]:
# datapath = 'Outputs_ATL03_test/'
datapath = 'IS2data/'
filelist = [datapath+f for f in listdir(datapath) if isfile(join(datapath, f)) & ('.h5' in f)]
print('number of processed ATL03 granules: ' + str(len(filelist)))
for f in filelist: print(f)

# a function for reading in the relevant variables from ATL03 granules

In [ ]:
def read_atl03(filename, geoid_h=True):
    # reads in the necessary variables at photon rate from an .h5 file in a dataframe:
    # - lat: latitude
    # - xatc: along-track distance from the equator crossing
    # - h: elevation above the geoid (or above reference ellipsoid if geoid_h is set to False)
    
    # open file
    f = h5py.File(filename, 'r')
    
    # make dictionary for beam data to be stored in
    dfs = {}
    dfs_bckgrd = {}
    beamlist = [x for x in list(f.keys()) if 'gt' in x]
    
    conf_landice = 3 # index for the land ice confidence
    
    orient = f['orbit_info']['sc_orient'][0]
    def orient_string(sc_orient):
        if sc_orient == 0:
            return 'backward'
        elif sc_orient == 1:
            return 'forward'
        elif sc_orient == 2:
            return 'transition'
        else:
            return 'error'
        
    orient_str = orient_string(orient)
    gtl = ['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']
    beam_strength_dict = {k:['weak','strong'][k%2] for k in np.arange(1,7,1)}
    if orient_str == 'forward':
        bl = np.arange(6,0,-1)
        gtx_beam_dict = {k:v for (k,v) in zip(gtl,bl)}
        gtx_strength_dict = {k:beam_strength_dict[gtx_beam_dict[k]] for k in gtl}
    elif orient_str == 'backward':
        bl = np.arange(1,7,1)
        gtx_beam_dict = {k:v for (k,v) in zip(gtl,bl)}
        gtx_strength_dict = {k:beam_strength_dict[gtx_beam_dict[k]] for k in gtl}
    else:
        gtx_beam_dict = {k:'undefined' for k in gtl}
        gtx_strength_dict = {k:'undefined' for k in gtl}
        

    ancillary = {'atlas_sdp_gps_epoch': f['ancillary_data']['atlas_sdp_gps_epoch'][0],
                 'rgt': f['orbit_info']['rgt'][0],
                 'cycle_number': f['orbit_info']['cycle_number'][0],
                 'sc_orient': orient_str,
                 'gtx_beam_dict': gtx_beam_dict,
                 'gtx_strength_dict': gtx_strength_dict}
    
    # loop through all beams
    for beam in beamlist:
        try:
            #### get photon-level data
            df = pd.DataFrame({'lat': np.array(f[beam]['heights']['lat_ph']),
                               'lon': np.array(f[beam]['heights']['lon_ph']),
                               'h': np.array(f[beam]['heights']['h_ph']),
                               'dt': np.array(f[beam]['heights']['delta_time']),
                               # 'conf': np.array(f[beam]['heights']['signal_conf_ph'][:,conf_landice]),
                               # not using ATL03 confidences here
                               'mframe': np.array(f[beam]['heights']['pce_mframe_cnt']),
                               'qual': np.array(f[beam]['heights']['quality_ph'])}) 
                               # 0=nominal,1=afterpulse,2=impulse_response_effect,3=tep

            df_bckgrd = pd.DataFrame({'pce_mframe_cnt': np.array(f[beam]['bckgrd_atlas']['pce_mframe_cnt']),
                                      'bckgrd_counts': np.array(f[beam]['bckgrd_atlas']['bckgrd_counts']),
                                      'bckgrd_int_height': np.array(f[beam]['bckgrd_atlas']['bckgrd_int_height']),
                                      'delta_time': np.array(f[beam]['bckgrd_atlas']['delta_time'])})

            #### calculate along-track distances [meters from the equator crossing] from segment-level data
            df['xatc'] = np.full_like(df.lat, fill_value=np.nan)
            ph_index_beg = np.int32(f[beam]['geolocation']['ph_index_beg']) - 1
            segment_dist_x = np.array(f[beam]['geolocation']['segment_dist_x'])
            segment_length = np.array(f[beam]['geolocation']['segment_length'])
            valid = ph_index_beg>=0 # need to delete values where there's no photons in the segment (-1 value)

            df.loc[ph_index_beg[valid], 'xatc'] = segment_dist_x[valid]
            df.xatc.fillna(method='ffill',inplace=True)
            df.xatc += np.array(f[beam]['heights']['dist_ph_along'])

            #### now we can filter out TEP (we don't do IRF / afterpulses because it seems to not be very good...)
            df.query('qual < 3',inplace=True) 
            # df.drop(columns=['qual'], inplace=True)

            #### sort by along-track distance (for interpolation to work smoothly)
            df.sort_values(by='xatc',inplace=True)
            df.reset_index(inplace=True, drop=True)

            if geoid_h:
                #### interpolate geoid to photon level using along-track distance, and add to elevation
                geophys_geoid = np.array(f[beam]['geophys_corr']['geoid'])
                print(np.sum(np.isnan(df.xatc)))
                print(np.sum(np.isnan(geophys_geoid)))
                print(np.sum(np.isnan(segment_dist_x+0.5*segment_length)))
                geoid = np.interp(np.array(df.xatc), segment_dist_x+0.5*segment_length, geophys_geoid)
                df['h'] = df.h - geoid
                df['geoid'] = geoid

            #### save to list of dataframes
            dfs[beam] = df
            dfs_bckgrd[beam] = df_bckgrd
            print(beam, end=' ')
        except Exception as e:
            print('Error for {f:s} on {b:s} ... skipping:'.format(f=filename, b=beam), e)
    return dfs, dfs_bckgrd, ancillary

# read in the data

In [ ]:
# for ifile, file in enumerate(filelist):
for ifile, file in enumerate([filelist[0]]):
    print('Reading file {i:4d} / {n:4d}: {fn:s} >>> '.format(i=ifile+1, n=len(filelist), fn=file[file.find('ATL03_'):]), end = ' ')
    photon_data, bckgrd_data, ancillary = read_atl03(file)
    print('')

In [ ]:
ancillary

In [ ]:
ancillary['cycle_number'].dtype

# a list of known lakes for development / checking results

In [ ]:
class KnownLake:
    def __init__(self, fn, gtx, latlims):
        self.fn = fn
        self.gtx = gtx
        self.latlims = latlims
    
    def getData(self, dfs):
        df = dfs[self.gtx]
        return df[(df.lat > self.latlims[0]) & (df.lat < self.latlims[1])]

In [ ]:
fn = file # to keep this out of the loop for now
lakes = {1: KnownLake(fn, 'gt1l', [67.433, 67.463]),
         2: KnownLake(fn, 'gt1l', [68.6, 68.65]),
         3: KnownLake(fn, 'gt1l', [68.535, 68.595]),
         4: KnownLake(fn, 'gt1l', [68.353, 68.37]),
         5: KnownLake(fn, 'gt1l', [68.02, 68.09]),
         6: KnownLake(fn, 'gt1l', [67.95, 67.98]),
         7: KnownLake(fn, 'gt1l', [67.675, 67.6975]),
         8: KnownLake(fn, 'gt1l', [67.37, 67.4]),
         9: KnownLake(fn, 'gt1l', [67.02, 67.055]),
         10: KnownLake(fn, 'gt2l', [68.716, 68.726]),
         11: KnownLake(fn, 'gt2l', [68.5, 68.547]),
         12: KnownLake(fn, 'gt2l', [68.33, 68.375]),
         13: KnownLake(fn, 'gt2l', [68.2425, 68.27]),
         14: KnownLake(fn, 'gt2l', [67.9023, 67.938]),
         15: KnownLake(fn, 'gt2l', [67.617, 67.671]),
         16: KnownLake(fn, 'gt2l', [67.3827, 67.4151]),
         17: KnownLake(fn, 'gt2l', [67.0057, 67.0408]),
         18: KnownLake(fn, 'gt2r', [67.3826, 67.4171]),
         19: KnownLake(fn, 'gt3l', [68.5505, 68.5826]),
         20: KnownLake(fn, 'gt3l', [68.4309, 68.4508]),
         21: KnownLake(fn, 'gt3l', [67.07, 67.1]),
         22: KnownLake(fn, 'gt3l', [66.9306, 67.0137]),
         23: KnownLake(fn, 'gt2l', [-71.6687, -71.6166]),
         24: KnownLake(fn, 'gt2l', [-71.8903, -71.8587]),
         25: KnownLake(fn, 'gt2l', [-72.9505, -72.8595]),
         26: KnownLake(fn, 'gt1l', [-72.6468, -72.5127]),
         27: KnownLake(fn, 'gt3l', [-73.2389, -73.2073]),
         28: KnownLake(fn, 'gt1l', [-70.85, -70.745]),
         29: KnownLake(fn, 'gt1l', [-71.513, -71.445]),
         30: KnownLake(fn, 'gt1l', [-72.94, -72.85]),
         31: KnownLake(fn, 'gt3l', [-72.8602, -72.6714]),
         32: KnownLake(fn, 'gt3l', [-72.08, -71.99]),
         33: KnownLake(fn, 'gt1l', [66, 69]),
         34: KnownLake(fn, 'gt2l', [66, 69]),
         35: KnownLake(fn, 'gt3l', [66, 69]),
         36: KnownLake(fn, 'gt1r', [66, 69]),
         37: KnownLake(fn, 'gt2r', [66, 69]),
         38: KnownLake(fn, 'gt3r', [66, 69]),
         39: KnownLake(fn, 'gt1l', [-74, -70.5]),
         40: KnownLake(fn, 'gt2l', [-74, -70.5]),
         41: KnownLake(fn, 'gt3l', [-74, -70.5]),
         42: KnownLake(fn, 'gt1r', [-74, -70.5]),
         43: KnownLake(fn, 'gt2r', [-74, -70.5]),
         44: KnownLake(fn, 'gt3r', [-74, -70.5]),
         45: KnownLake(fn, 'gt1l', [63, 72]),
         46: KnownLake(fn, 'gt1l', [-90, 90]),
         47: KnownLake(fn, 'gt2l', [-90, 90]),
         48: KnownLake(fn, 'gt3l', [-90, 90]),
         49: KnownLake(fn, 'gt1r', [-90, 90]),
         50: KnownLake(fn, 'gt2r', [-90, 90]),
         51: KnownLake(fn, 'gt3r', [-90, 90]),
        }

# make a DataFrame for a given granule/gtx

In [ ]:
%%time
thisLake = 46
df = lakes[thisLake].getData(photon_data)
df['snr'] = 0.0
plt.close('all')

# create a dataframe that aggregates info at the major frame level

In [ ]:
%%time
mframe_group = df.groupby('mframe')
df_mframe = mframe_group[['lat','lon', 'xatc', 'dt']].mean()
df_mframe.drop(df_mframe.head(1).index,inplace=True)
df_mframe.drop(df_mframe.tail(1).index,inplace=True)
def convert_time_to_string(dt):
    epoch = dt + datetime.datetime.timestamp(datetime.datetime(2018,1,1))
    return datetime.datetime.fromtimestamp(epoch).strftime("%Y-%m-%d, %H:%M:%S")
df_mframe['time'] = df_mframe['dt'].map(convert_time_to_string)
df_mframe['xatc_min'] = mframe_group['xatc'].min()
df_mframe['xatc_max'] = mframe_group['xatc'].max()
df_mframe['n_phot'] = mframe_group['h'].count()

# check for flat surfaces in each major frame of the granule

In [ ]:
%%time
def find_flat_lake_surfaces(mframe, df=df, bin_height_coarse=1.0, bin_height_fine=0.02, smoothing_histogram=0.2, buffer=4.0,
                            width_surf=0.1, width_buff=0.3, rel_dens_upper_thresh=0.3, rel_dens_lower_thresh=0.3):
    selector_segment = (df.mframe == mframe)
    dfseg = df[selector_segment]
    
    # find main broad peak
    bins_coarse1 = np.arange(start=dfseg.h.min(), stop=dfseg.h.max(), step=bin_height_coarse)
    hist_mid1 = bins_coarse1[:-1] + 0.5 * bin_height_coarse
    peak_loc1 = hist_mid1[np.argmax(np.histogram(dfseg.h, bins=bins_coarse1)[0])]
    
    # decrease bin width and find finer peak
    bins_coarse2 = np.arange(start=peak_loc1-buffer, stop=peak_loc1+buffer, step=bin_height_fine)
    hist_mid2 = bins_coarse2[:-1] + 0.5 * bin_height_fine
    hist = np.histogram(dfseg.h, bins=bins_coarse2)
    window_size = int(smoothing_histogram/bin_height_fine)
    hist_vals = hist[0] / np.max(hist[0])
    hist_vals_smoothed = np.array(pd.Series(hist_vals).rolling(window_size,center=True,min_periods=1).mean())
    peak_loc2 = hist_mid2[np.argmax(hist_vals_smoothed)]
    
    # calculate relative photon densities
    peak_upper = peak_loc2 + width_surf
    peak_lower = peak_loc2 - width_surf
    above_upper = peak_upper + width_buff
    below_lower = peak_lower - width_buff
    sum_peak = np.sum((dfseg.h > peak_lower) & (dfseg.h < peak_upper))
    sum_above = np.sum((dfseg.h > peak_upper) & (dfseg.h < above_upper))
    sum_below = np.sum((dfseg.h > below_lower) & (dfseg.h < peak_lower))
    rel_dens_upper = (sum_above / width_buff) / (sum_peak / (width_surf*2))
    rel_dens_lower = (sum_below / width_buff) / (sum_peak / (width_surf*2))
    
    # check for flat surface, if found calculate SNR and look for bottom return
    is_flat_like_lake = (rel_dens_upper < rel_dens_upper_thresh) & (rel_dens_lower < rel_dens_lower_thresh)
    
    return peak_loc2, is_flat_like_lake

# get all the flat segments and select
df_mframe['peak'], df_mframe['is_flat'] = list(zip(*df_mframe.index.map(find_flat_lake_surfaces)))

# a function for calculating SNR and checking for second returns
(apply this only to the major frames with flat surfaces for now)

In [ ]:
# initialize columns to be added
df_mframe['lake_qual_pass'] = False
df_mframe['has_densities'] = False
df_mframe['ratio_2nd_returns'] = 0.0
df_mframe['quality_summary'] = 0.0
df_mframe['h_2nd_returns'] = [[]] * len(df_mframe)
df_mframe['xatc_2nd_returns'] = [[]] * len(df_mframe)
df_mframe['proms_2nd_returns'] = [[]] * len(df_mframe)
df_mframe_selected = df_mframe[df_mframe['is_flat']]

def get_densities_and_2nd_peaks(df, df_mframe, df_mframe_selected, aspect=30, K_phot=10, dh_signal=0.2, n_subsegs=7,
                                bin_height_snr=0.1, smoothing_length=1.0, buffer=4.0, print_results=False):
    
    for mframe in df_mframe_selected.index:
        
        selector_segment = (df.mframe == mframe)
        dfseg = df[selector_segment]

        xmin = df_mframe_selected['xatc_min'].loc[mframe]
        xmax = df_mframe_selected['xatc_max'].loc[mframe]
        nphot = df_mframe_selected['n_phot'].loc[mframe]
        peak_loc2 = df_mframe_selected['peak'].loc[mframe]

        # the radius in which to look for neighbors
        dfseg_nosurface = dfseg[(dfseg.h > (peak_loc2+dh_signal)) | (dfseg.h < (peak_loc2-dh_signal))]
        nphot_bckgrd = len(dfseg_nosurface.h)
        # radius of a circle in which we expect to find one non-lake-surface signal photon
        wid = np.sqrt(((dfseg_nosurface.h.max()-dfseg_nosurface.h.min()-2*dh_signal)*((xmax-xmin)/aspect)/nphot_bckgrd)/np.pi)

        # buffer segment for density calculation
        selector_buffer = (df.xatc >= (dfseg.xatc.min()-aspect*wid)) & (df.xatc <= (dfseg.xatc.max()+aspect*wid))
        dfseg_buffer = df[selector_buffer]
        dfseg_buffer.xatc += np.random.uniform(low=-0.35, high=0.35, size=len(dfseg_buffer.xatc))

        # normalize xatc to be regularly spaced and scaled by the aspect parameter
        xmin_buff = dfseg_buffer.xatc.min()
        xmax_buff = dfseg_buffer.xatc.max()
        nphot_buff = len(dfseg_buffer.xatc)
        xnorm = np.linspace(xmin_buff, xmax_buff, nphot_buff) / aspect

        # KD tree query distances
        Xn = np.array(np.transpose(np.vstack((xnorm, dfseg_buffer['h']))))
        kdt = KDTree(Xn, leaf_size=40)
        idx, dist = kdt.query_radius(Xn, r=wid, count_only=False, return_distance=True,sort_results=True)
        density = (np.array([np.sum(1-np.abs(x/wid)) if (len(x)<(K_phot+1)) 
                   else np.sum(1-np.abs(x[:K_phot+1]/wid))
                   for x in dist]) - 1) / K_phot

        #print(' density calculated')
        densities = np.array(density[dfseg_buffer.mframe == mframe])
        densities /= np.max(densities)

        # add SNR to dataframes
        dfseg['snr'] = densities
        df.loc[selector_segment, 'snr'] = dfseg['snr']
        df_mframe['has_densities'].loc[mframe] = True

        # subdivide into segments again to check for second return
        subsegs = np.linspace(xmin, xmax, n_subsegs+1) 
        subsegwidth = subsegs[1] - subsegs[0]

        n_2nd_returns = 0
        prominences = []
        elev_2ndpeaks = []
        subpeaks_xatc = []
        for subsegstart in subsegs[:-1]:

            subsegend = subsegstart + subsegwidth
            selector_subseg = ((dfseg.xatc > subsegstart) & (dfseg.xatc < subsegend))
            dfsubseg = dfseg[selector_subseg]
            
            # avoid looking for peaks when there's no / very little data
            if len(dfsubseg > 10):

                # get the median of the snr values in each bin
                bins_subseg_snr = np.arange(start=np.max((dfsubseg.h.min(),peak_loc2-70)), stop=peak_loc2+2*buffer, step=bin_height_snr)
                mid_subseg_snr = bins_subseg_snr[:-1] + 0.5 * bin_height_snr
                snrstats = binned_statistic(dfsubseg.h, dfsubseg.snr, statistic='median', bins=bins_subseg_snr)
                snr_median = snrstats[0]
                snr_median[np.isnan(snr_median)] = 0
                window_size_sub = int(smoothing_length/bin_height_snr)
                snr_vals_smoothed = np.array(pd.Series(snr_median).rolling(window_size_sub,center=True,min_periods=1).mean())
                snr_vals_smoothed /= np.max(snr_vals_smoothed)

                # take histogram binning values into account, but clip surface peak to second highest peak height
                subhist, subhist_edges = np.histogram(dfsubseg.h, bins=bins_subseg_snr)
                subhist_nosurface = subhist.copy()
                subhist_nosurface[(mid_subseg_snr < (peak_loc2+2*dh_signal)) & (mid_subseg_snr > (peak_loc2-2*dh_signal))] = 0
                subhist_nosurface_smoothed = np.array(pd.Series(subhist_nosurface).rolling(window_size_sub,center=True,min_periods=1).mean())
                subhist_max = subhist_nosurface_smoothed.max()
                subhist_smoothed = np.array(pd.Series(subhist).rolling(window_size_sub,center=True,min_periods=1).mean())
                subhist_smoothed = np.clip(subhist_smoothed, 0, subhist_max)
                subhist_smoothed /= np.max(subhist_smoothed)

                # combine histogram and snr values to find peaks
                snr_hist_smoothed = subhist_smoothed * snr_vals_smoothed
                peaks, peak_props = find_peaks(snr_hist_smoothed, height=0.1, distance=int(0.5/bin_height_snr), prominence=0.1)

                if len(peaks) >= 2: 
                    idx_surfpeak = np.argmin(peak_loc2 - mid_subseg_snr[peaks])
                    peak_props['prominences'][idx_surfpeak] = 0

                    # classify as second peak only if prominence is larger 0.2
                    prominence_secondpeak = np.max(peak_props['prominences'])
                    prominence_threshold = 0.3
                    if prominence_secondpeak > prominence_threshold:

                        idx_2ndreturn = np.argmax(peak_props['prominences'])
                        secondpeak_h = mid_subseg_snr[peaks[idx_2ndreturn]]

                        # classify as second peak only if elevation is 0.5m lower than main peak (surface) and higher than 50m below surface
                        if (secondpeak_h < (peak_loc2-0.5)) & (secondpeak_h > (peak_loc2-50)):
                            secondpeak_xtac = subsegstart + subsegwidth/2
                            n_2nd_returns += 1
                            prominences.append(prominence_secondpeak)
                            elev_2ndpeaks.append(secondpeak_h)
                            subpeaks_xatc.append(secondpeak_xtac)

        ratio_2nd_returns = n_2nd_returns/n_subsegs
        df_mframe['ratio_2nd_returns'].loc[mframe] = ratio_2nd_returns
        quality_secondreturns = np.sum(prominences) / n_subsegs

        min_quality = (0.2 + (ratio_2nd_returns-0.2)*(0.1/0.8))
        quality_summary = 0.0
        quality_pass = 'No'
        if (ratio_2nd_returns >= 0.2) & (quality_secondreturns > min_quality):
            quality_pass = 'Yes'
            quality_summary = ratio_2nd_returns*(quality_secondreturns-min_quality)/(ratio_2nd_returns-min_quality)
            df_mframe['lake_qual_pass'].loc[mframe] = True
            df_mframe['quality_summary'].loc[mframe] = quality_summary
            df_mframe['h_2nd_returns'].loc[mframe] = elev_2ndpeaks
            df_mframe['xatc_2nd_returns'].loc[mframe] = subpeaks_xatc
            df_mframe['proms_2nd_returns'].loc[mframe] = prominences
        # if (percent_2d_returns >= 30) & (quality_secondreturns > 0.4):
        flatstring = 'Yes' if df_mframe['is_flat'].loc[mframe] else 'No'
        
        if print_results:
            print('mframe %d: Elevation: %.2fm. Flat: %s. 2nd returns %3d%%. Quality summary: %.2f. Passes check: %s.' % \
                (mframe, peak_loc2, flatstring, np.round(ratio_2nd_returns*100), quality_summary, quality_pass))

# calculate the densities and possible second returns

In [ ]:
%%time
get_densities_and_2nd_peaks(df, df_mframe, df_mframe_selected, print_results=False)

# check for densities and possible second returns two major frames around where high-quality lakes were detected

In [ ]:
%%time
# check for densities and possible second returns two mframes around where passing quality lakes were detected
num_additional = 1
count = 0
while (num_additional > 0) & (count<20):
    count+=1
    selected = df_mframe[df_mframe['lake_qual_pass']].index 
    lst1 = np.unique(list(selected-2) + list(selected-1) + list(selected+1) + list(selected+2))
    lst2 = df_mframe.index
    inter = list(set(lst1) & set(lst2))
    df_include_surrounding = df_mframe.loc[inter]
    # df_include_surrounding = df_mframe.loc[np.unique(list(selected-2) + list(selected-1) + list(selected+1) + list(selected+2))]
    df_additional = df_include_surrounding[~df_include_surrounding['has_densities']]
    num_additional = len(df_additional)
    if num_additional > 0:
        get_densities_and_2nd_peaks(df, df_mframe, df_additional, print_results=False)

# merge detected lakes iteratively

In [ ]:
%%time
max_dist_mframes = 7
max_dist_elev = 0.1
df_mframe.sort_index(inplace=True)
start_mframe = np.array(df_mframe.index[df_mframe['lake_qual_pass']],dtype=int)
stop_mframe = np.array(df_mframe.index[df_mframe['lake_qual_pass']],dtype=int)
surf_elevs = np.array(df_mframe['peak'][df_mframe['lake_qual_pass']])
n_lakes = len(surf_elevs)
n_lakes_old = n_lakes + 1
iteration = 0

debug = False

while n_lakes < n_lakes_old:
    
    start_mframe_old = start_mframe
    stop_mframe_old = stop_mframe
    surf_elevs_old = surf_elevs
    n_lakes_old = n_lakes
    start_mframe = []
    stop_mframe = []
    surf_elevs = []
    
    last_merged = True
    for i in range(n_lakes - 1):
        is_closeby = ((start_mframe_old[i + 1] - stop_mframe_old[i]) <= max_dist_mframes)
        is_at_same_elevation = (np.abs(surf_elevs_old[i + 1] - surf_elevs_old[i]) < max_dist_elev)
        if debug: 
            print('%10d %10d diff: %4d, close: %5s, %7.2f %7.2f diff: %7.2f, same: %5s' % \
                 (stop_mframe_old[i], start_mframe_old[i + 1], start_mframe_old[i + 1] - stop_mframe_old[i],
                  is_closeby, surf_elevs_old[i], surf_elevs_old[i + 1], np.abs(surf_elevs_old[i + 1] - surf_elevs_old[i]),
                  is_at_same_elevation), end=' ')
        
        # if merging
        if (is_closeby & is_at_same_elevation):
            
            if last_merged == False:
                start_mframe[-1] = start_mframe_old[i]
                stop_mframe[-1] = stop_mframe_old[i + 1]
                surf_elevs[-1] = (surf_elevs_old[i] + surf_elevs_old[i+1]) / 2
            else: 
                start_mframe.append(start_mframe_old[i])
                stop_mframe.append(stop_mframe_old[i + 1])
                surf_elevs.append((surf_elevs_old[i] + surf_elevs_old[i+1]) / 2)
            if debug:
                print('--> merge')
            last_merged = True
        
        # if keeping 
        elif (iteration > 0):
            
            if i==0:
                start_mframe.append(start_mframe_old[i])
                stop_mframe.append(stop_mframe_old[i])
                surf_elevs.append(surf_elevs_old[i])
            
            start_mframe.append(start_mframe_old[i+1])
            stop_mframe.append(stop_mframe_old[i+1])
            surf_elevs.append(surf_elevs_old[i+1])
            if debug:
                print('--> keep')
            last_merged = False
    
    iteration += 1
    n_lakes = len(surf_elevs)
    print('iteration %3d, number of lakes: %4d' % (iteration, n_lakes))
    
df_extracted_lakes = pd.DataFrame({'mframe_start': start_mframe, 'mframe_end': stop_mframe, 'surf_elev': surf_elevs})
df_extracted_lakes

# check for adjacient peaks at surface elevation to extend lake further

In [ ]:
n_check = 3
elev_tol = 0.2
for i in range(len(df_extracted_lakes)):
    
    thislake = df_extracted_lakes.iloc[i]
    thiselev = thislake['surf_elev']
    
    # check for extending before
    extent_before = thislake['mframe_start']
    check_before = extent_before - 1
    left_to_check = n_check
    while left_to_check > 0:
        # print('check!')
        if np.abs(df_mframe['peak'].loc[check_before] - thiselev) < elev_tol:
            extent_before = check_before
            left_to_check = n_check
            print('extend before')
        else:
            left_to_check -= 1
            
        check_before -= 1
        
    df_extracted_lakes['mframe_start'].iloc[i] = extent_before
    
    # check for extending after
    extent_after = thislake['mframe_end']
    check_after = extent_after + 1
    left_to_check = n_check
    while left_to_check > 0:
        # print('check!')
        if np.abs(df_mframe['peak'].loc[check_after] - thiselev) < elev_tol:
            extent_after = check_after
            left_to_check = n_check
            print('extend after')
        else:
            left_to_check -= 1
            
        check_after += 1
        
    df_extracted_lakes['mframe_end'].iloc[i] = extent_after

# expand each lake by two major frames
df_extracted_lakes['mframe_start'] -= 2
df_extracted_lakes['mframe_end'] += 2
df_extracted_lakes
    # for mf in check_before

# run the SNR algorithm and second peak finding for the remaining major frames within lake boundaries

In [ ]:
dfs_to_calculate_densities = []

for i in range(len(df_extracted_lakes)):
    
    thislake = df_extracted_lakes.iloc[i]
    extent_start = thislake['mframe_start']
    extent_end = thislake['mframe_end']
    
    dfs_to_calculate_densities.append(df_mframe[(df_mframe.index >= extent_start) & (df_mframe.index <= extent_end)])
    
df_to_calculate_densities = pd.concat(dfs_to_calculate_densities)
df_to_calculate_densities = df_to_calculate_densities[~df_to_calculate_densities['has_densities']]

get_densities_and_2nd_peaks(df, df_mframe, df_to_calculate_densities, print_results=False)

# plot for checking

In [ ]:
plt.close('all')

for i in range(len(df_extracted_lakes)):
# for i in [2]:
    
    thislake = df_extracted_lakes.iloc[i]
    thiselev = thislake['surf_elev']
    extent_start = thislake['mframe_start']
    extent_end = thislake['mframe_end']
    
    # subset the dataframes to the current lake extent
    df_lake = df[(df['mframe'] >= extent_start) & (df['mframe'] <= extent_end)]
    df_mframe_lake = df_mframe[(df_mframe.index >= extent_start) & (df_mframe.index <= extent_end)]
    h_2nds = [v for l in list(df_mframe_lake['h_2nd_returns']) for v in l]
    xatc_2nds = [v for l in list(df_mframe_lake['xatc_2nd_returns']) for v in l]
    prom_2nds = [v for l in list(df_mframe_lake['proms_2nd_returns']) for v in l]
    
    # get statistics
    # average mframe quality summary excluding the two mframes for buffer on each side
    lake_quality = np.sum(df_mframe_lake['quality_summary']) / (len(df_mframe_lake) - 4)
    lake_time = convert_time_to_string(df_mframe_lake['dt'].mean())
    lake_lat = df_mframe_lake['lat'].mean()
    lake_lat_min = df_mframe_lake['lat'].min()
    lake_lat_max = df_mframe_lake['lat'].max()
    lake_lat_str = '%.5f°N'%(lake_lat) if lake_lat>=0 else '%.5f°S'%(-lake_lat)
    lake_lon = df_mframe_lake['lon'].mean()
    lake_lon_min = df_mframe_lake['lon'].min()
    lake_lon_max = df_mframe_lake['lon'].max()
    lake_lon_str = '%.5f°E'%(lake_lon) if lake_lon>=0 else '%.5f°W'%(-lake_lon)
    lake_gtx = lakes[thisLake].gtx
    lake_track = ancillary['rgt']
    lake_cycle = ancillary['cycle_number']
    lake_beam_strength = ancillary['gtx_strength_dict'][lake_gtx]
    lake_minh = np.min((df_mframe_lake['peak'].min(), np.min(h_2nds)))
    h_range = thiselev - lake_minh
    lake_maxh = np.min((df_mframe_lake['peak'].max(), thiselev+5*h_range))
    buffer_top = np.max((0.2*h_range, 1.0))
    buffer_bottom = np.max((0.3*h_range, 2.0))
    lake_minh_plot = lake_minh - buffer_bottom
    lake_maxh_plot = lake_maxh + buffer_top
    
    fig = plt.figure(figsize=[9, 5], dpi=100)
    ax = fig.add_subplot(111)
       
    ax.scatter(df_lake.xatc-df_lake.xatc.min(), df_lake.h, s=6, c='k', alpha=0.05, edgecolors='none')
    scatt = ax.scatter(df_lake.xatc-df_lake.xatc.min(), df_lake.h, s=3, c=df_lake.snr, alpha=1, edgecolors='none',
                       cmap=cmc.lajolla,vmin=0,vmax=1)
                        
    # plot surface elevation
    xmin, xmax = ax.get_xlim()
    ax.plot([xmin, xmax], [thiselev, thiselev], 'g-', lw=0.5)
    
    # plot mframe bounds
    ymin, ymax = ax.get_ylim()
    mframe_bounds_xatc = list(df_mframe_lake['xatc_min']) + [df_mframe_lake['xatc_max'].iloc[-1]]
    for xmframe in mframe_bounds_xatc:
        ax.plot([xmframe-df_lake.xatc.min(), xmframe-df_lake.xatc.min()], [ymin, ymax], 'k-', lw=0.5)

    dfpass = df_mframe_lake[df_mframe_lake['lake_qual_pass']]
    dfnopass = df_mframe_lake[~df_mframe_lake['lake_qual_pass']]
    ax.plot(dfpass.xatc-df_lake.xatc.min(), dfpass.peak, marker='o', mfc='g', mec='g', linestyle = 'None', ms=5)
    ax.plot(dfnopass.xatc-df_lake.xatc.min(), dfnopass.peak, marker='o', mfc='none', mec='r', linestyle = 'None', ms=3)

    for i,prom in enumerate(prom_2nds):
        ax.plot(xatc_2nds[i]-df_lake.xatc.min(), h_2nds[i], marker='o', mfc='none', mec='b', linestyle = 'None', ms=prom*4)

    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='4%', pad=0.05)
    fig.colorbar(scatt, cax=cax, orientation='vertical')
    
    ax.set_ylim((lake_minh_plot, lake_maxh_plot))
    ax.set_xlim((0.0, df_mframe_lake['xatc_max'].iloc[-1] - df_lake.xatc.min()))
    
    ax.set_title('Lake at (%s, %s) on %s\nICESat-2 track %d %s (%s), cycle %d [lake quality: %.2f]' % \
                 (lake_lat_str, lake_lon_str, lake_time, lake_track, lake_gtx,lake_beam_strength, lake_cycle, lake_quality))
    ax.set_ylabel('elevation above geoid [m]')
    ax.set_xlabel('along-track distance [m]')
    
    # ax.text(0.99,0.01,'lat')
    url = 'https://openaltimetry.org/data/api/icesat2/atlXX?'
    url += 'date={date}&minx={minx}&miny={miny}&maxx={maxx}&maxy={maxy}&trackId={track}&beamName={beam}'.format(
            date=lake_time[:10],minx=lake_lon_min,miny=lake_lat_min,maxx=lake_lon_max,maxy=lake_lat_max,track=lake_track,beam=lake_gtx)
    url += '&outputFormat=json&client=jupyter'
    print('oaurl = \'%s\'' % url)
    
    # save figure
    fig_dir = 'figs/lakes_found_test/'
    if not os.path.exists(fig_dir): os.makedirs(fig_dir)
    epoch = df_mframe_lake['dt'].mean() + datetime.datetime.timestamp(datetime.datetime(2018,1,1))
    dateid = datetime.datetime.fromtimestamp(epoch).strftime("%Y%m%d-%H%M%S")
    latid = '%dN'%(int(np.round(lake_lat*1e5))) if lake_lat>=0 else '%dS'%(-int(np.round(lake_lat*1e5)))
    lonid = '%dE'%(int(np.round(lake_lon*1e5))) if lake_lon>=0 else '%dS'%(-int(np.round(lake_lon*1e5)))
    figname = fig_dir + 'lake_found_%s%s_%d_%s_%s-%s.jpg' % (lake_track, lake_gtx, lake_cycle, dateid, latid, lonid)
    plt.savefig(figname, dpi=600, bbox_inches='tight', pad_inches=0)

In [ ]:
# !zip -r figs/lakes_found_test.zip figs/lakes_found_test